# Evaluation of the model

## Set up paths

In [ ]:
import os, sys

MAIN_DIR = os.path.abspath("..").replace(' ','')
sys.path.insert(0,MAIN_DIR)

# Path to the exported model to be evaluated. Change to use another model. 
PATH_TO_MODEL = os.path.normpath('/workspace/CabinetsCV/models/fine-tuned/efficient_det_all_v8')

# Path to the folder that contains images and their ground truth annotations used for evaluation. 
# Images (.jpg) and annotations (.xml) are expected to have the same names.
PATH_TO_TEST_IMG_DIR = os.path.normpath('/workspace/CabinetsCV/data/test_images_all')

#\models\fine-tuned\efficient_det_1024_rand_aug_1_4\exported
SAVED_MODEL = os.path.join(PATH_TO_MODEL, 'exported', 'saved_model')
# Label map used during training
PATH_TO_LABELMAP = os.path.normpath('/workspace/CabinetsCV/data/train-valid-split/label_map.pbtxt') 

In [ ]:
import time
import random
import matplotlib.pyplot as plt
from pathlib import Path
import tensorflow as tf

# our modules in src/
from src.evaluation import *
import src.visualize

import warnings
# suppress Matplotlib warnings
warnings.filterwarnings('ignore')   
%matplotlib inline

## Load saved model

In [ ]:
# Load the model and build the detection function
DETECT_FN = tf.saved_model.load(SAVED_MODEL)

## Test data preprocessing

In [ ]:
# We will resize test images to this target size as part of preprocessing
TARGET_IMAGE_SIZE = 1024
# How we reshape input images (see below)
RESHAPE_FLAG = 'scale_min_and_crop'

## Get detections on evaluation test images

In [ ]:
detections_from_dir = detections_from_image_dir(
    image_dir = PATH_TO_TEST_IMG_DIR,
    detect_fn = DETECT_FN,
    target_size = TARGET_IMAGE_SIZE,
    label_map_path =  PATH_TO_LABELMAP,
    min_score_thresh = 0.,
    reshape = RESHAPE_FLAG,
    only_top_left = False
    )

In [ ]:
get_coco_metrics(PATH_TO_TEST_IMG_DIR, detections_from_dir,
                 PATH_TO_LABELMAP,
                 target_size = TARGET_IMAGE_SIZE,
                 reshape = RESHAPE_FLAG,
                 include_metrics_per_category = False)

In [ ]:
MIN_SCORE_THRESH = 0.5
IOU_THRESH = 0.5

In [ ]:
confMetrics = get_metrics(PATH_TO_TEST_IMG_DIR,
                          detections_from_dir,
                          PATH_TO_LABELMAP,
                          min_score_thresh = MIN_SCORE_THRESH,
                          iou_thresh = IOU_THRESH)

In [ ]:
## OPTIONAL
# write confusion matrix contents to file in the main directory
fileName = PATH_TO_MODEL.replace('models/fine-tuned/','')+str('.txt')
f = open(fileName, "w+")

for it in confMetrics.items():
    f.write(str(it[0])+'\t'+str(it[1])+'\n')
    
f.close()
os.chmod(fileName,775)

## Print confusion matrix results

In [ ]:
src.visualize.print_metrics(confMetrics,
                            PATH_TO_TEST_IMG_DIR,
                            PATH_TO_LABELMAP,
                            min_score_thresh = MIN_SCORE_THRESH,
                            iou_thresh = IOU_THRESH)

## Visualize evaluation images

In [ ]:
NUM_IMG_PLOT = 85
NUM_IMG_ROW = 2

In [ ]:
image_names = [x for x in os.listdir(PATH_TO_TEST_IMG_DIR) if '.jpg' in x or '.jpeg' in x]
images_to_plot = sorted(random.sample(image_names, NUM_IMG_PLOT))

In [ ]:
src.visualize.plot_predictions_from_paths(
    images_to_plot,
    detections_from_dir,
    PATH_TO_LABELMAP,
    min_score_thresh=0.5,
    img_per_row = 2,
    linewidth = 8)

## Save evaluation images

In [ ]:
MIN_SCORE_THRESH = 0.5
TARGET_DIR = '/workspace/CabinetsCV/data/evaluation-images-v8'
if not Path(TARGET_DIR).exists():
    Path(TARGET_DIR).mkdir()
src.visualize.save_predictions(
                 TARGET_DIR,
                 images_to_plot,
                 detections_from_dir,
                 PATH_TO_LABELMAP,
                 min_score_thresh = MIN_SCORE_THRESH,
                 linewidth = 7
                )